# Task
Implement a real-time face emotion prediction system using a Convolutional Neural Network (CNN). The system should be able to:
1. Unzip and load an image dataset from `archive.zip`.
2. Preprocess the image data, including resizing, normalizing, and data augmentation.
3. Build, train, and evaluate a CNN model for face emotion classification.
4. Implement real-time emotion prediction from a webcam feed, detecting faces, preprocessing them, and displaying the predicted emotion.

## Unzip Dataset

### Subtask:
Extract the contents of the provided 'archive.zip' file to access the image dataset for face emotion recognition.


**Reasoning**:
To extract the contents of the archive.zip file, I will use the unzip command in a code block, as specified in the instructions.



In [ ]:
import zipfile
import os

zip_file_path = '/content/archive.zip'
extract_dir = '/content/emotion_dataset'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Archive '{zip_file_path}' unzipped to '{extract_dir}'")

# List the contents of the extracted directory to verify
print("Contents of the extracted directory:")
for root, dirs, files in os.walk(extract_dir):
    level = root.replace(extract_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

Archive '/content/archive.zip' unzipped to '/content/emotion_dataset'
Contents of the extracted directory:
emotion_dataset/
    train/
        disgust/
            Training_89374221.jpg
            Training_90061714.jpg
            Training_80443921.jpg
            Training_52480931.jpg
            Training_92587210.jpg
            Training_81867140.jpg
            Training_13740441.jpg
            Training_52302945.jpg
            Training_12896398.jpg
            Training_63156802.jpg
            Training_3566649.jpg
            Training_25421729.jpg
            Training_94788538.jpg
            Training_24397096.jpg
            Training_27695321.jpg
            Training_23533665.jpg
            Training_22751863.jpg
            Training_48988640.jpg
            Training_20847777.jpg
            Training_69173187.jpg
            Training_99142151.jpg
            Training_79594467.jpg
            Training_24546801.jpg
            Training_93510905.jpg
            Training_73989844.jpg

# Task
Implement a data loading and preprocessing pipeline for the image dataset extracted from `archive.zip`. This involves using `ImageDataGenerator` to load images, resize them to a uniform dimension (e.g., 48x48 pixels), normalize pixel values, and perform data augmentation (e.g., rotation, shifting, zooming). Finally, split the preprocessed data into training and validation sets.

## Load and Preprocess Data

### Subtask:
Load the extracted image dataset, preprocess the images (resize, normalize, augment), and split them into training and validation sets using ImageDataGenerator.


**Reasoning**:
The subtask requires loading and preprocessing the image data using `ImageDataGenerator`. This involves importing the necessary library, defining directory paths, configuring data augmentation for training, and setting up data generators for both training and validation sets.



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to training and validation data
train_dir = '/content/emotion_dataset/train'
validation_dir = '/content/emotion_dataset/test'

# Image parameters
IMG_HEIGHT = 48
IMG_WIDTH = 48
batch_size = 64
num_classes = 7 # Assuming 7 emotion classes based on common datasets

# Data augmentation and rescaling for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
validation_datagen = ImageDataGenerator(
    rescale=1./255
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    color_mode='grayscale', # Common for emotion recognition
    class_mode='categorical' # For multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    color_mode='grayscale', # Common for emotion recognition
    class_mode='categorical',
    shuffle=False # No need to shuffle validation data
)

print("Training data generator created successfully.")
print("Validation data generator created successfully.")


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Training data generator created successfully.
Validation data generator created successfully.


## Build CNN Model

### Subtask:
Design and build a Convolutional Neural Network (CNN) architecture suitable for face emotion classification.


**Reasoning**:
To design and build the CNN architecture, I will use `tensorflow.keras.models.Sequential` and various layers like `Conv2D`, `MaxPooling2D`, `Flatten`, `Dense`, and `Dropout` as specified in the instructions. Then, I will print the model summary to review the architecture.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize the Sequential model
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening the output for Dense layers
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Print the model summary
model.summary()

print("CNN model built successfully.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,847 (1.36 MB)

 Trainable params: 355,847 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

CNN model built successfully.


**Reasoning**:
The next logical step after building the CNN model is to compile it by defining the optimizer, loss function, and metrics, and then train it using the prepared data generators.



In [ ]:
import tensorflow as tf
import math

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
epochs = 50 # You can adjust this number

# Calculate steps per epoch and validation steps to ensure all samples are covered
steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = math.ceil(validation_generator.samples / validation_generator.batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

print("Model compiled and trained successfully.")

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 129s 281ms/step - accuracy: 0.5083 - loss: 1.2992 - val_accuracy: 0.5751 - val_loss: 1.1320
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 124s 275ms/step - accuracy: 0.5072 - loss: 1.2957 - val_accuracy: 0.5687 - val_loss: 1.1262
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 125s 278ms/step - accuracy: 0.5168 - loss: 1.2735 - val_accuracy: 0.5706 - val_loss: 1.1340
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 124s 275ms/step - accuracy: 0.5119 - loss: 1.2823 - val_accuracy: 0.5711 - val_loss: 1.1306
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 127s 283ms/step - accuracy: 0.5162 - loss: 1.2837 - val_accuracy: 0.5720 - val_loss: 1.1136
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 123s 275ms/step - accuracy: 0.5209 - loss: 1.2686 - val_accuracy: 0.5694 - val_loss: 1.1147
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 127s 282ms/step - accuracy: 0.5176 - loss: 1.2683 - val_accuracy: 0.5688 - val_loss: 1.1197
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 139s 275ms/step - accuracy: 0.5173 -

### Subtask:
Implement real-time emotion prediction from a webcam feed, detecting faces, preprocessing them, and displaying the predicted emotion.

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the saved model
model = load_model('emotion_detection_model.h5')

# Load OpenCV's pre-trained Haar cascade for face detection
# This file needs to be present in the Colab environment or mounted
# You might need to download it: !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

def detect_and_predict_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            prediction = model.predict(roi)[0]
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'No Face', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return frame

print("Model loaded and face classifier initialized. Ready for real-time prediction (requires webcam feed).")
print("Note: For actual real-time webcam functionality, this code would typically run in a local environment with webcam access.")


Model loaded and face classifier initialized. Ready for real-time prediction (requires webcam feed).
Note: For actual real-time webcam functionality, this code would typically run in a local environment with webcam access.


## Implement Real-time Prediction

### Subtask:
Save the trained CNN model for future use in real-time prediction.

In [ ]:
# Save the trained model
model_save_path = 'emotion_detection_model.h5'
model.save(model_save_path)
print(f"Model saved successfully to {model_save_path}")

Model saved successfully to emotion_detection_model.h5
